In [1]:
# Execute if necessary
# %%capture
# !pip install numpy seaborn matplotlib pandas

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List

# Práctica 3: Modelo Lineal

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones. Cada función está marcada por &#x1F625;,  &#x1F643; o &#x1F921;. Las marcas indican:

- &#x1F625;: Indican una entrega que debe ser hecha dentro de la misma sesión de la asignación. 
- &#x1F643;: Indican una entrega que puede ser hecha hasta la siguiente sesión.
- &#x1F921;: Debe mostrar un avance en la misma sesión, pero la entrega puede ser hecha en la siguiente.

Aquellas entregas parciales que no sean hechas el día de la asignación ya no serán válidas para las entregas totales, sin embargo, las entregas totales seguirán siendo válidas.

En esta sección se incluye un dataset real. El dataset importado incluye multiples características que describen las condiciones de los pasajeros en el accidente del titanic.

- __PassengerId__: Identificador de cada pasajero.
- __Survived__: 0 si no sobrevivió al accidente, 1 si lo hizo.
- __Pclass__: Clase en la que viajaba el pasajero, 1 - Primera clase, 2 - Segunda clase y 3 - Tercera clase.
- __Name__: Nombre del pasajero.
- __Sex__: Sexo del pasajero.
- __Age__: Edad del pasajero.
- __SibSp__: Número de hermanos más número de esposas con las que viajaba el pasajero.
- __Parch__: Número de padres más número de hijos con las que viajaba el pasajero.
- __Ticket__: Número de boleto.
- __Fare__: Tarifa del boleto del pasajero.
- __Cabin__: Número de cabina del pasajero.
- __Embarked__: Puerto de embarcación, C - Cherbourg, Q - Queenstown y S - Southampton.


In [3]:
df = pd.read_csv("titanic.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Asignación 1 &#x1F625;

Realice el preprocesamiento que considere adecuado para que las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__, __Embarked__ y __Survived__ puedan ser utilizadas por un modelo lineal.

In [4]:
# Obtenemos los atributos deseados
new_df = df[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']]

### One Hot Encoder 

In [5]:
# Utilizamos un One Hot Encoder para los atributos 'Sex' 
ohe_sex = pd.get_dummies(new_df['Sex'], prefix = 'Sex')
new_df = new_df.drop('Sex', axis = 1)
new_df = new_df.join(ohe_sex)

In [6]:
# Utilizamos un One Hot Encoder para los atributos'Embarked'
ohe_embarked = pd.get_dummies(new_df['Embarked'], prefix = 'Embarked')
new_df = new_df.drop('Embarked', axis = 1)
new_df = new_df.join(ohe_embarked)

### Obtención de nuevos resultados

In [7]:
# Observamos las transformaciones en los atributos
new_df.dtypes

Pclass          int64
SibSp           int64
Parch           int64
Fare          float64
Survived        int64
Sex_female      uint8
Sex_male        uint8
Embarked_C      uint8
Embarked_Q      uint8
Embarked_S      uint8
dtype: object

In [8]:
# Imprimimos los primeros 10 registros
new_df.head(10)

,Pclass,SibSp,Parch,Fare,Survived,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0,7.2500,0,0,1,0,0,1
1,1,1,0,71.2833,1,1,0,1,0,0
2,3,0,0,7.9250,1,1,0,0,0,1
3,1,1,0,53.1000,1,1,0,0,0,1
4,3,0,0,8.0500,0,0,1,0,0,1
5,3,0,0,8.4583,0,0,1,0,1,0
6,1,0,0,51.8625,0,0,1,0,0,1
7,3,3,1,21.0750,0,0,1,0,0,1
8,3,0,2,11.1333,1,1,0,0,0,1
9,2,1,0,30.0708,1,1,0,1,0,0


In [9]:
def calc_skewness(x) -> float:
    """
    Descripción:
    Esta funcion calcula el skewness de un atributo.
    
    Entradas:
    - x: Los samples de un atributo dado. 
            
    Salidas:
    - El cálculo de skewness.
    """
    return (((x-x.mean()) / x.var()**(1/2)) ** 3).mean()

In [10]:
def transform_atributes(dataset, attributes):
    for i in attributes:
        skewness_i = calc_skewness(dataset[i])
        print('Skewness Atributo {}:'.format(i), skewness_i)
        dataset[i] = dataset[i] ** (1 / 2)
        dataset[i] = (dataset[i] - dataset[i].mean()) / dataset[i].std()
        skewness_i = calc_skewness(dataset[i])
        print('Skewness Nuevo Atributo {}:'.format(i), skewness_i)
    return dataset

In [11]:
new_df = transform_atributes(new_df, ['Fare', 'SibSp', 'Parch'])

Skewness Atributo Fare: 4.771209669373592
Skewness Nuevo Atributo Fare: 2.0779894539935944
Skewness Atributo SibSp: 3.682918775041469
Skewness Nuevo Atributo SibSp: 1.4316933042143463
Skewness Atributo Parch: 2.739867686513194
Skewness Nuevo Atributo Parch: 1.5246520886312336


In [12]:
new_df.head(10)

,Pclass,SibSp,Parch,Fare,Survived,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1.003846,-0.539444,-0.732705,0,0,1,0,0,1
1,1,1.003846,-0.539444,1.219137,1,1,0,1,0,0
2,3,-0.629184,-0.539444,-0.691106,1,1,0,0,0,1
3,1,1.003846,-0.539444,0.826767,1,1,0,0,0,1
4,3,-0.629184,-0.539444,-0.683600,0,0,1,0,0,1
5,3,-0.629184,-0.539444,-0.659479,0,0,1,0,1,0
6,1,-0.629184,-0.539444,0.797776,0,0,1,0,0,1
7,3,2.199307,1.299098,-0.088411,0,0,1,0,0,1
8,3,-0.629184,2.060647,-0.514086,1,1,0,0,0,1
9,2,1.003846,-0.539444,0.214677,1,1,0,1,0,0


## Asignación 2 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir __Survived__ utilizando el algoritmo _pocket_. Imprima el error obtenido.

In [13]:
# Obtenemos X
X = new_df.drop('Survived', axis = 1).to_numpy()
# Obtenemos y
y = new_df['Survived'].to_numpy()

In [32]:
class PocketLinearClassifier:
    
    def __init__(self, learning_rate = 0.24, max_iterations = 100):
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        
    def fit(self, X, y):
        # Agregamos una columna con valor uno a la matriz X
        X = np.insert(X, 0, 1, axis = 1)
        # Inicializamos un vector de ceros
        w = np.zeros(X.shape[1])
        # Inicializamos el mejor vecotr de pesos y le menor error de clasificación
        best_w = np.copy(w)
        best_error = len(X)
        # Iteramos el máximos nuemero de iteraciones
        for i in range(self.max_iterations):
            # Calculamos las clasificaciones predichas
            y_pred = np.sign(np.dot(X, w))
            y_pred[y_pred == -1] = 0
            # Calculamos el error de clasificación
            equals = np.array([1 if y_pred[i] == y[i] else 0 for i in range(len(y))])
            error = equals.sum() / len(X)
            print("Error iteración {}:".format(i), error)
            if error < best_error:
                best_w = np.copy(w)
                best_error = error
            # Si no hay errore, terminamos el entrenamiento
            if error == 0:
                break
            # Calculamos el gradiente
            gradiente = -np.dot((y - y_pred), X)
            # Actualizamos el vector de pesos con el gradiente y el factor de aprendizaje
            w = w - self.learning_rate * gradiente
        self.best_error = best_error
        self.coef_ = best_w
        return self
    
    def predict(self, X):
        X = np.insert(X, 0, 1, axis = 1)
        # Calculamos las clasifiaciones 
        y_pred = np.sign(np.dot(X, self.coef_))
        y_pred[y_pred == -1 ] = 0
        y_pred[y_pred == 1] = 1
        return y_pred

In [33]:
# Generamos nuestro objeto para entrenar
plr = PocketLinearClassifier(max_iterations = 1000)

In [34]:
# Entrenamos el clasificador
plr.fit(X, y)

Error iteración 0: 0.6161616161616161
Error iteración 1: 0.3838383838383838
Error iteración 2: 0.6318742985409652
Error iteración 3: 0.7295173961840629
Error iteración 4: 0.7800224466891134
Error iteración 5: 0.7070707070707071
Error iteración 6: 0.6778900112233446
Error iteración 7: 0.3838383838383838
Error iteración 8: 0.6184062850729517
Error iteración 9: 0.7160493827160493
Error iteración 10: 0.4051627384960718
Error iteración 11: 0.6408529741863075
Error iteración 12: 0.7261503928170595
Error iteración 13: 0.7643097643097643
Error iteración 14: 0.7811447811447811
Error iteración 15: 0.7744107744107744
Error iteración 16: 0.7845117845117845
Error iteración 17: 0.7643097643097643
Error iteración 18: 0.7934904601571269
Error iteración 19: 0.6958473625140292
Error iteración 20: 0.7182940516273849
Error iteración 21: 0.3838383838383838
Error iteración 22: 0.6206509539842873
Error iteración 23: 0.7171717171717171
Error iteración 24: 0.7160493827160493
Error iteración 25: 0.8002244668911

Error iteración 289: 0.7721661054994389
Error iteración 290: 0.7946127946127947
Error iteración 291: 0.755331088664422
Error iteración 292: 0.8013468013468014
Error iteración 293: 0.4388327721661055
Error iteración 294: 0.6565656565656566
Error iteración 295: 0.7586980920314254
Error iteración 296: 0.7542087542087542
Error iteración 297: 0.7732884399551067
Error iteración 298: 0.7991021324354658
Error iteración 299: 0.7508417508417509
Error iteración 300: 0.7867564534231201
Error iteración 301: 0.40291806958473625
Error iteración 302: 0.6206509539842873
Error iteración 303: 0.7418630751964085
Error iteración 304: 0.755331088664422
Error iteración 305: 0.7744107744107744
Error iteración 306: 0.7957351290684624
Error iteración 307: 0.7508417508417509
Error iteración 308: 0.7822671156004489
Error iteración 309: 0.41414141414141414
Error iteración 310: 0.6262626262626263
Error iteración 311: 0.7508417508417509
Error iteración 312: 0.755331088664422
Error iteración 313: 0.7755331088664422
E

Error iteración 588: 0.7485970819304153
Error iteración 589: 0.3838383838383838
Error iteración 590: 0.6161616161616161
Error iteración 591: 0.7261503928170595
Error iteración 592: 0.7530864197530864
Error iteración 593: 0.77665544332211
Error iteración 594: 0.7901234567901234
Error iteración 595: 0.7822671156004489
Error iteración 596: 0.7833894500561167
Error iteración 597: 0.7822671156004489
Error iteración 598: 0.7676767676767676
Error iteración 599: 0.7811447811447811
Error iteración 600: 0.6868686868686869
Error iteración 601: 0.7171717171717171
Error iteración 602: 0.3838383838383838
Error iteración 603: 0.6161616161616161
Error iteración 604: 0.7362514029180696
Error iteración 605: 0.7182940516273849
Error iteración 606: 0.8080808080808081
Error iteración 607: 0.7508417508417509
Error iteración 608: 0.8002244668911336
Error iteración 609: 0.44556677890011226
Error iteración 610: 0.632996632996633
Error iteración 611: 0.7519640852974186
Error iteración 612: 0.7418630751964085
Er

Error iteración 889: 0.43658810325476993
Error iteración 890: 0.6240179573512907
Error iteración 891: 0.7485970819304153
Error iteración 892: 0.734006734006734
Error iteración 893: 0.797979797979798
Error iteración 894: 0.77665544332211
Error iteración 895: 0.7856341189674523
Error iteración 896: 0.7497194163860831
Error iteración 897: 0.7878787878787878
Error iteración 898: 0.388327721661055
Error iteración 899: 0.6161616161616161
Error iteración 900: 0.7171717171717171
Error iteración 901: 0.7474747474747475
Error iteración 902: 0.7744107744107744
Error iteración 903: 0.7991021324354658
Error iteración 904: 0.7508417508417509
Error iteración 905: 0.7991021324354658
Error iteración 906: 0.43434343434343436
Error iteración 907: 0.6307519640852974
Error iteración 908: 0.7575757575757576
Error iteración 909: 0.7384960718294051
Error iteración 910: 0.7867564534231201
Error iteración 911: 0.7991021324354658
Error iteración 912: 0.755331088664422
Error iteración 913: 0.7968574635241302
Erro

In [35]:
plr.best_error

0.3838383838383838

In [ ]:
# Predeicmos
y_pred = plr.predict(X)

In [ ]:
# Generamos las tablas
from tabulate import tabulate
table = [[y[i], y_pred[i]] for i in range(len(y))]
headers =  ['y', 'y_pred']
print(tabulate(table, headers, tablefmt = "github"))

## Asignación 3 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Cabin__ y __Embarked__, entrene una regresión lineal para predecir __Fare__ utilizando el algoritmo de Ordinary Leasts Squares (OLS). Imprima el valor del error cuadrático medio (MSE).

In [53]:
# Obtenemos X
X = new_df.drop('Survived', axis = 1)
X = X.drop('Fare', axis = 1).to_numpy()
# Obtenemos y
y = new_df['Fare'].to_numpy()

In [54]:
class OrdinartLeastsSquares:
    
    def fit(self, X, y):
        #Agregamos una columna con valor uno a la matriz X
        X = np.insert(X, 0, 1, axis = 1)
        # coef, res, _, __ = np.linalg.lstsq(X, y, rcond = None)
        # self.coef_ = coef
        XTX = X.T @ X
        XTXinv = np.linalg.inv(XTX)
        pseudo_inv = XTXinv @ X.T
        self.coef_ = pseudo_inv @ y
        return self
    
    def predict(self, X):
        X = np.insert(X, 0, 1, axis = 1)
        # Realizamos las predicciones
        y_pred = X.dot(self.coef_)
        return y_pred
    
    def mse(self, y, y_pred):
        diff = y - y_pred
        diff = diff ** 2
        mse_result = diff.mean()
        return mse_result

In [55]:
ols = OrdinartLeastsSquares()

In [56]:
ols.fit(X, y)

In [57]:
ols.coef_

array([-43.18054921,  -0.76889858,   0.19415525,   0.19278544,
        44.18247225,  43.42900309,  -0.3209706 ,  -0.54460355,
        -0.66035991])

In [58]:
y_pred = ols.predict(X)

In [59]:
y_pred

array([-2.62769689,  0.00295876, -2.19128905, -0.33643056, -2.94475821,
       -2.82900184, -1.40696104, -2.04114764, -1.69002919, -0.76593983,
       -1.51978348, -0.65349188, -2.94475821, -1.83513546, -2.19128905,
       -1.42239047, -1.84043495, -2.17585962, -1.87422773, -1.85189974,
       -2.17585962, -2.17585962, -2.07553269, -1.40696104, -1.28767849,
       -1.0816663 , -2.60536889, -0.35653486, -2.07553269, -2.94475821,
       -1.06757172,  0.00295876, -2.07553269, -2.17585962, -0.7505104 ,
       -1.08989972, -2.60536889, -2.94475821, -1.74289663, -1.53483841,
       -1.87422773, -1.10532914, -2.60536889, -0.26467997, -2.07553269,
       -2.94475821, -2.51194052, -2.07553269, -2.15697647, -1.87422773,
       -1.95619132, -2.94475821,  0.00295876, -1.10532914, -0.71312748,
       -1.40696104, -1.42239047, -2.60536889, -0.60406929, -1.73452768,
       -2.60536889,  0.00686803, -1.08989972, -1.89433203, -1.06757172,
       -1.93386332, -1.42239047, -2.94475821, -1.05590655, -2.49

In [60]:
ols.mse(y, y_pred)

3.909917549524275

In [61]:
# Generamos las tablas
from tabulate import tabulate
table = [[y[i], y_pred[i]] for i in range(len(y))]
headers =  ['y', 'y_pred']
print(tabulate(table, headers, tablefmt = "github"))

|           y |      y_pred |
|-------------|-------------|
| -0.732705   | -2.6277     |
|  1.21914    |  0.00295876 |
| -0.691106   | -2.19129    |
|  0.826767   | -0.336431   |
| -0.6836     | -2.94476    |
| -0.659479   | -2.829      |
|  0.797776   | -1.40696    |
| -0.0884105  | -2.04115    |
| -0.514086   | -1.69003    |
|  0.214677   | -0.76594    |
| -0.259547   | -1.51978    |
|  0.102321   | -0.653492   |
| -0.6836     | -2.94476    |
|  0.25158    | -1.83514    |
| -0.695384   | -2.19129    |
| -0.288929   | -1.42239    |
|  0.185172   | -1.84043    |
| -0.422817   | -2.17586    |
| -0.20657    | -1.87423    |
| -0.734282   | -1.8519     |
|  0.0841106  | -2.17586    |
| -0.422817   | -2.17586    |
| -0.684845   | -2.07553    |
|  0.375738   | -1.40696    |
| -0.0884105  | -1.28768    |
|  0.254991   | -1.08167    |
| -0.734282   | -2.60537    |
|  3.85798    | -0.356535   |
| -0.693872   | -2.07553    |
| -0.692868   | -2.94476    |
|  0.140468   | -1.06757    |
|  2.462  

## Asignación 4 &#x1F921;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir la probabilidad de supervivencia __Survived__ utilizando el algoritmo de gradiente descendente estocástico y la entropía cruzada como función de error. Imprima el arror en cada iteración del gradiente.

In [ ]:
X = np.array(new_df.drop(columns = ['Survived']))
y = np.array(new_df['Survived'])

In [ ]:
def E_in_log_reg_sgd(X, y, w):
    acum_sum = 0
    for i in range(X.shape[0]):
        acum_sum += np.log(1 + np.exp(X[i] @ w * y[i]))
    
    return acum_sum / X.shape[0]

In [ ]:
def sigmoid(x):
    return np.exp(x) / (1 + np.exp(x))

In [ ]:
def cross_entropy_error(X, y, w):
    acum_sum = 0
    for i in range(X.shape[0]):
        acum_sum += (match_results(y[i], 1) * np.log(1 / sigmoid(X[i] @ w))) + (match_results(y[i], -1) * np.log(1 / (1 - sigmoid(X[i] @ w))))
    
    return acum_sum / X.shape[0]

In [ ]:
def __random_fn_def__():
    def __random_fn__(shape):
        return np.zeros(shape)
    
    return __random_fn__
    
__random_fn__ = __random_fn_def__()

In [ ]:
def match_results(a, b):
    if a != b:
        return 1
    else:
        return 0

In [ ]:
def logistic_reg_sgd(X, y, lr, T = 100):
    N = X.shape[0]
    acum_sum = 0
    w_0 = __random_fn__((X.shape[1]))
    w = list()
    w.append(w_0)
    for i in range(T):
        print("Itera {}".format(i))
        indexes = [ind for ind in range(N)]
        badge_indexes = np.random.choice(indexes, 1, False)
        for j in badge_indexes:
            acum_sum += (y[j] * X[j]) / (1 + np.exp(y[j] * X[j] @ w[-1]))
        g_t = (- 1 / len(badge_indexes)) * acum_sum
        print(g_t)
        v_t = -1 * g_t
        w_next = w[-1] + lr * v_t
        print(w_next)
        w.append(w_next)
        acum_sum = 0
        e_in = E_in_log_reg_sgd(X, y, w_next)
        #print(e_in)
        cee = cross_entropy_error(X, y, w_next)
        print(cee)
    return w[-1]

In [ ]:
logistic_reg_sgd(X, y, 0.01, 1000)